In [1]:
import tensorflow as tf
import numpy as np
from sdm_ml.dataset import BBSDataset
from sklearn.preprocessing import StandardScaler
from block_diag import block_diagonal
from kernels import ard_rbf_kernel
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
csv_folder = '../../multi_species/bbs/dataset/csv_bird_data/'
dataset = BBSDataset(csv_folder)

In [3]:
training_set = dataset.get_training_set()

In [4]:
training_set.keys()

dict_keys(['covariates', 'outcomes'])

In [5]:
training_set['outcomes'].sum().sort_values(ascending=False).head()

Mourning Dove           1356
American Robin          1341
Red-winged Blackbird    1283
Brown-headed Cowbird    1229
American Crow           1224
dtype: int64

In [6]:
training_set['outcomes'].shape

(1612, 371)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

covariates = scaler.fit_transform(training_set['covariates'].values)
outcomes = training_set['outcomes'][['American Crow', 'Brown-headed Cowbird']]

scaler = StandardScaler()

# Keep only the first 200 rows and 2 outcomes for this test
covariates = scaler.fit_transform(covariates.astype(np.float32))[:200, :]
outcomes = outcomes.values[:200, :1]

/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
covariates.shape

(200, 8)

In [9]:
n_features = covariates.shape[1]
n_outcomes = outcomes.shape[1]
kern_size = covariates.shape[0] * n_outcomes

In [10]:
from single_output_laplace_gp import SingleOutputLaplaceGP

In [11]:
gp = SingleOutputLaplaceGP(n_features=covariates.shape[1],
                           n_data=covariates.shape[0])

In [12]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

In [13]:
def to_optimise(hypers):
    
    scales = hypers[:-1]
    alpha = hypers[-1]
    
    results = gp.marg_lik_and_grad(scales=scales,
                                   alpha=alpha,
                                   X=covariates,
                                   y=outcomes[:, 0],
                                   sess=sess)
    
    return results

In [14]:
marg_lik = lambda x: -to_optimise(x)['log_marg_lik']

def grad(hypers):
    
    print(hypers)
    
    opt_results = to_optimise(hypers)
    
    combined_grad = np.concatenate([opt_results['scale_grad'],
                                    np.reshape(opt_results['alpha_grad'], (-1,))])
        
    return -combined_grad

In [15]:
from scipy.optimize import minimize

results = minimize(marg_lik, x0=np.linspace(2, 8, 8).tolist() + [2.],
                   jac=grad, tol=1e-7)

[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 222
  Number of functions evaluations: 248
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of itera

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iteratio

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iteratio

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iterations: 1
  Number of functions evaluations: 17
[2.         2.85714286 3.71428571 4.57142857 5.42857143 6.28571429
 7.14285714 8.         2.        ]
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 78.187981
  Number of iteratio

In [16]:
results

      fun: 88.92854
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]])
      jac: array([  4.0465794 ,   2.6522546 ,   2.326644  ,   0.22869308,
         0.44901812,   0.47156855,   1.6606867 ,   0.6604654 ,
       -11.932709  ], dtype=float32)
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 64
      nit: 0
     njev: 52
   status: 2
  success: False
        x: array([2.        , 2.85714286, 3.71428571, 4.57142857, 5.42857143,
       6.28571429, 7.14285714, 8.        , 2.        ])